In [ ]:
%tensorflow_version 2.x
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

print(tf.__version__)

import math

# get some random numbers
X = 10.0 * np.random.rand(20000)
X = pd.DataFrame(data=X)

# save some for testing
x_train = X.sample(frac=0.8)
x_test = X.drop(x_train.index)

# generate the ground truth
y_train = x_train.apply(math.sin, axis=1)
y_test = x_test.apply(math.sin, axis=1)

print(x_train, y_train)


In [ ]:

fig = plt.figure(figsize=(9,9))
plt.ylim([-2, 2])
plt.scatter(x_train, y_train)

In [ ]:
#--------MODEL BUILDING
num_params = len(x_train.keys())
print(num_params)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([num_params], name="Input_Layer"),
    tf.keras.layers.Dense(64, activation='relu', name="dense_01"),
    tf.keras.layers.Dense(64, activation='relu', name="dense_02"),
    tf.keras.layers.Dense(64, activation='relu', name="dense_03"),
    # 1 node in the output for the sin(x)
    tf.keras.layers.Dense(1, name="Output_Layer")
  ])

learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate),
              # loss function to minimize
              loss='mae',
              # list of metrics to monitor
              metrics=['mae',])

model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=True,
    dpi=96,
)

In [ ]:
# Fit/Train model on training data
history = model.fit(x_train, y_train,
                    batch_size=20,
                    epochs=10,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
#--------MONITOR
# Plot training & validation loss values
fig = plt.figure(figsize=(12,9))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')
plt.show()

In [ ]:
#--------EVALUATE
loss, mae = model.evaluate(x_test, y_test, verbose=2)
print('Loss:', loss, 'MAE:', mae)

In [ ]:
#--------PREDICT
p_test = model.predict(x_test)

fig = plt.figure(figsize=(9,9))
plt.ylim([-2, 2])
plt.scatter(x_test, p_test)
plt.scatter(x_test, y_test)


In [ ]:
#----------PLOT True Values vs. Predictions
fig = plt.figure(figsize=(9,9))
a = plt.axes(aspect='equal')
plt.scatter(y_test, p_test)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 1]
plt.xlim(lims)
plt.ylim(lims)
# draw a diagonal line for comparison
plt.plot(lims, lims)
plt.show()

In [ ]:
#---------PLOT the distribution of errors
fig = plt.figure(figsize=(9,9))
error = p_test.flatten() - y_test
plt.hist(error, bins = 5)
plt.xlabel("Prediction Error")
plt.ylabel("Count")
plt.show()